In [1]:
import persistence_homology as ph

In [2]:
import csv
import os
import json
from pathlib import Path
import logging

In [3]:
log_path = Path("notebook_demo.log")
logging.basicConfig(
    filename='automation.log',         # log file path
    filemode='a',                  # 'w' to overwrite each run, 'a' to append
    format='%(asctime)s %(levelname)-8s %(name)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO             # minimum level to record
)
logger = logging.getLogger(__name__)

logger.info("Notebook logging started")

In [4]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output
def get_subfolders(path):
    """
    Return a list of names of all subdirectories in the given path.
    """
    return [
        name for name in os.listdir(path)
        if os.path.isdir(os.path.join(path, name))
    ]
def write_json(json_file_name, data):
    output_path = Path("{}.json".format(json_file_name))
    with output_path.open("w", encoding="utf-8") as f:
        json.dump(
            data,            # Python object
            f,               # file handle
            ensure_ascii=False,  # keep unicode chars if any
            indent=4             # pretty-print with 4-space indent
        )

In [5]:
segmentation_folder_path = "./lung_segmentations"
lung_segmentations = get_subfolders(segmentation_folder_path)

In [6]:
print(lung_segmentations)

['Lung segmentation-AI-25', 'Lung segmentation-AI-48', 'Lung segmentation-AI-15', 'Lung segmentation-56', 'Lung segmentation-4', 'Lung segmentation-34', 'Lung segmentation-57', 'Lung segmentation-2', 'Lung segmentation-32', 'Lung segmentation-19', 'Lung segmentation-10', 'Lung segmentation-AI-9', 'Lung segmentation-17', 'Lung segmentation-20', 'Lung segmentation-AI-39', 'Lung segmentation-AI-36', 'Lung segmentation-AI-19', 'Lung segmentation-AI-17', 'Lung segmentation-AI-10', 'Lung segmentation-AI-29', 'Lung segmentation-AI-20', 'Lung segmentation-39', 'Lung segmentation-9', 'Lung segmentation-52', 'Lung segmentation-36', 'Lung segmentation-65', 'Lung segmentation-AI-4', 'Lung segmentation-25', 'Lung segmentation-48', 'Lung segmentation-15', 'Lung segmentation-AI-2', 'Lung segmentation-AI-34', 'Lung segmentation-AI-32']


# Filtration Algorithm Start

In [9]:
for lung_segmentation in lung_segmentations:
    print(lung_segmentation)

Lung segmentation-AI-25
Lung segmentation-AI-48
Lung segmentation-AI-15
Lung segmentation-56
Lung segmentation-4
Lung segmentation-34
Lung segmentation-57
Lung segmentation-2
Lung segmentation-32
Lung segmentation-19
Lung segmentation-10
Lung segmentation-AI-9
Lung segmentation-17
Lung segmentation-20
Lung segmentation-AI-39
Lung segmentation-AI-36
Lung segmentation-AI-19
Lung segmentation-AI-17
Lung segmentation-AI-10
Lung segmentation-AI-29
Lung segmentation-AI-20
Lung segmentation-39
Lung segmentation-9
Lung segmentation-52
Lung segmentation-36
Lung segmentation-65
Lung segmentation-AI-4
Lung segmentation-25
Lung segmentation-48
Lung segmentation-15
Lung segmentation-AI-2
Lung segmentation-AI-34
Lung segmentation-AI-32


In [10]:
for lung_segmentation in lung_segmentations:
    folder_path = './lung_segmentations/{}'.format(lung_segmentation)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    vertices = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    print(len(edges))

67551
108234
79816
50826
78781
92825
84330
138334
55377
123279
77781
73461
119766
45903
78654
98091
123279
119766
77781
63121
45903
78654
74136
103843
98091
85228
78781
67500
108681
79816
138334
92825
55437


In [6]:
json_data = {}
for lung_segmentation in lung_segmentations:
    folder_path = './lung_segmentations/{}'.format(lung_segmentation)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    vertices = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    directions = ph.generate_sphere_points(20, 5, 0.0000001)
    directions = [[0,0,1]]
    print("Lung Segmentation-{}:".format(lung_segmentation))
    segmentation_data = []
    for direction in directions:
        bz = ph.BettiZero(direction, vertices, edges)
        components, mergers, verts, births = bz.compute_persistence()
        intervals = ph.compute_intervals(births, mergers)
        largest_bar = ph.compute_largest_bar(intervals)
        data = {
            "direction": direction,
            "intervals": intervals,
            "largest_bar": largest_bar,
            "components": list(components)
        }
        segmentation_data.append(data)
        print("Direction: {}. Largest Bar: {}.".format(direction, largest_bar))
        logger.info(data)
    json_data[lung_segmentation] = segmentation_data
write_json("BettiZeroSegmentations", json_data)

Lung Segmentation-Lung segmentation-AI-19:
Direction: [0, 0, 1]. Largest Bar: (78.0, [130.5, 208.5]).
Lung Segmentation-Lung segmentation-AI-2:
Direction: [0, 0, 1]. Largest Bar: (93.0, [68.5, 161.5]).
Lung Segmentation-Lung segmentation-25:
Direction: [0, 0, 1]. Largest Bar: (72.0, [97.5, 169.5]).
Lung Segmentation-Lung segmentation-20:
Direction: [0, 0, 1]. Largest Bar: (57.0, [98.5, 155.5]).
Lung Segmentation-Lung segmentation-AI-32:
Direction: [0, 0, 1]. Largest Bar: (63.0, [86.5, 149.5]).
Lung Segmentation-Lung segmentation-52:
Direction: [0, 0, 1]. Largest Bar: (46.0, [130.5, 176.5]).
Lung Segmentation-Lung segmentation-48:
Direction: [0, 0, 1]. Largest Bar: (89.0, [101.5, 190.5]).
Lung Segmentation-Lung segmentation-AI-10:
Direction: [0, 0, 1]. Largest Bar: (56.0, [114.5, 170.5]).
Lung Segmentation-Lung segmentation-AI-25:
Direction: [0, 0, 1]. Largest Bar: (72.0, [97.5, 169.5]).
Lung Segmentation-Lung segmentation-AI-17:
Direction: [0, 0, 1]. Largest Bar: (65.0, [72.5, 137.5]).

In [7]:
print(json_data)

{'Lung segmentation-AI-19': [{'direction': [0, 0, 1], 'intervals': [[107.5, -1], [110.5, 124.5], [114.5, 115.5], [118.5, 120.5], [129.5, 154.5], [129.5, 130.5], [129.5, -1], [130.5, 208.5], [131.5, 141.5], [131.5, 132.5], [132.5, 136.5], [133.5, 138.5], [135.5, 139.5], [137.5, 158.5], [138.5, 139.5], [140.5, 158.5], [141.5, 145.5], [146.5, 163.5], [146.5, 157.5], [147.5, 151.5], [148.5, 149.5], [149.5, 150.5], [149.5, 182.5], [150.5, 151.5], [151.5, 155.5], [152.5, 154.5], [157.5, -1], [157.5, 163.5], [159.5, -1], [160.5, 163.5], [161.5, 166.5], [164.5, 177.5], [164.5, 166.5], [164.5, 180.5], [165.5, 166.5], [166.5, 169.5], [174.5, -1], [176.5, 178.5], [178.5, 180.5], [180.5, 181.5], [180.5, 181.5], [180.5, 181.5], [181.5, 182.5], [181.5, 182.5], [182.5, 183.5], [188.5, 189.5], [189.5, 191.5], [194.5, -1], [210.5, 212.5], [210.5, 211.5], [210.5, -1], [211.5, 212.5], [212.5, -1], [216.5, 221.5]], 'largest_bar': (78.0, [130.5, 208.5]), 'components': [0, 711, 5908, 6503, 9760, 19473, 2434